**SUMMARY:** *In the previous lab, we used the TVLA to assess our usual AES implementation. In this lab, we'll see how a separate module, cwtvla, can make this even easier.*

**LEARNING OUTCOMES:**
* Using cwtvla repeat the TVLA from the previous lab
* Using cwtvla for additional TVLA testing

## CWTVLA

`cwtvla` has two main modules: an independent ktp/analysis module, and one designed for easy use with ChipWhisperer.
The first can be imported as follows:

In [ ]:
import cwtvla

And the second by:

In [ ]:
import cwtvla.cw_convenience as conv

In [ ]:
cw.__file__

`conv` here includes a function for automatically setting up a target. `"STM32F3"` will require a file called `AES.hex` in this directory with the firmware to load:

In [ ]:
import chipwhisperer as cw
scope, target = conv.setup_device("CW305", fpga_id='100t')

In [ ]:
target.fpga_read(0x09, 16)

cwtvla includes its own KTP classes. They function similarly to normal ChipWhisperer KTPs, with the major difference being that they include `next_group_A()` and `next_group_B()` methods for getting key text pairs for both groups whenever you want.

**NOTE:** RAMBUS's TVLA document suggests that capture of the two groups be randomly intersperced; however, the original TVLA specification only specifies that they should be intersperced. The following uses the latter spec.

In [ ]:
ktp = cwtvla.FixedVRandomText()
import chipwhisperer as cw
import numpy as np
from tqdm.autonotebook import trange
N = 5000
groupA = np.zeros((N, scope.adc.samples), dtype='float64')
groupB = np.zeros((N, scope.adc.samples), dtype='float64')

for i in trange(N):
    key, text = ktp.next_group_A()
    trace = cw.capture_trace(scope, target, text, key)
    groupA[i,:] = trace.wave[:]
    
    key, text = ktp.next_group_B()
    trace = cw.capture_trace(scope, target, text, key)
    groupB[i, :] = trace.wave[:]

In [ ]:
import chipwhisperer as cw
trace = cw.capture_trace(scope, target, text, key)
print(trace)

In [ ]:
group1, group2 = conv.capture_non_specific(scope, target, cwtvla.FixedVRandomText, N=1000)

In [ ]:
cw.plot(scope.get_last_trace(), label='test')

We can then run the T-Test as follows:

In [ ]:
t_val = cwtvla.t_test(group1, group2)
cw.plot(scope.get_last_trace()*100) * cw.plot(t_val[0]) * cw.plot(t_val[1])

cwtvla can also be used to report where the test failed at. For our software implementation, this will be many locations:

In [ ]:
fail_points = cwtvla.check_t_test(t_val)
print(fail_points)

`conv` also includes a function to automate collection of the TVLA data. We can use it as follows, using the fixed vs. random key test instead of the fixed vs. random text:

In [ ]:
group1, group2 = conv.capture_non_specific(scope, target, cwtvla.FixedVRandomKey, N=2000)

In [ ]:
t_val = cwtvla.t_test(group1, group2)
fail_points = cwtvla.check_t_test(t_val)
cw.plot(scope.get_last_trace()*60) * cw.plot(t_val[0]) * cw.plot(t_val[1]) 

In [ ]:
if len(fail_points) > 0:
    print("Test failed at {}".format(fail_points))

You should see much different results for this TVLA, though it still easily fails.

Let's look at the last non-specific TVLA included with `cwtvla`, semi fixed vs. random text:

In [ ]:
group1, group2 = conv.capture_non_specific(scope, target, cwtvla.SemiFixedVRandomText, N=10000)

In [ ]:
t_val = cwtvla.t_test(group1, group2)
fail_points = cwtvla.check_t_test(t_val)
cw.plot(scope.get_last_trace()*60) * cw.plot(t_val[0]) * cw.plot(t_val[1])